<font size="+4">Alexa, how many people have your name?</font>

***Work in Progress***

# Introduction

## Datasets

**Amazon Alexa Release dates [[1](#references)]:**

The release dates of Amazon Alexa for the different countries are covered by a table on the Wikipedia. I manually looked up the dates for the individual countries.

**Baby Names, England and Wales [[2](#references)] [[3](#references)]:**

The GB Office for National Statistics provided their data in two different subsets: The first dataset covers the years 1907 to 1994, but contains only the top 100 names and only their gender and rank, not their count. The second dataset covers from 1996 to 2018 is more complete, covering much more names (with count >= 3) and provides gender, rank and counts. I wasn't able to identify datasets for the missing year 1995.

**Baby Names, United States [[4](#references)]:**

This dataset provided by the US Social Security Administration (SSA) seems quite complete: It contains information on name, gender, count and rank covers for every years from 1880 to 2018. The fact that it contains names with single occurcance (count=1) indicates, that *all* names are included.

**Other possible Sources:**

- https://www.bfs.admin.ch/bfs/de/home/statistiken/bevoelkerung/geburten-todesfaelle/vornamen-neugeborene.html
- https://www.dia.govt.nz/diawebsite.nsf/wpg_URL/Services-Births-Deaths-and-Marriages-Most-Popular-Male-and-Female-First-Names
- https://data.sa.gov.au/data/dataset/popular-baby-names

# Setup

## Imports

In [1]:
# Default
from pathlib import Path
from zipfile import ZipFile
import io
import urllib
import datetime
import sys
import platform
import psutil

# Extra
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

pd.plotting.register_matplotlib_converters()
sns.set(style="darkgrid")

## System information

Sometimes useful for reproducability:

In [2]:
print("---- System information ----")
print(f"OS: {platform.uname().system}, {platform.uname().release}")
print(f"Platform: {platform.uname().machine}")
print("Logical cores:", psutil.cpu_count(logical=True))
print(f"Max Frequency: {psutil.cpu_freq().max:.0f} Mhz")
print(f"Total Memory: {psutil.virtual_memory().total / 1024 / 1024 / 1024:.2F} GB")

print("\n---- Python information ----")
print(f"python: {sys.version.split(' ')[0]}")
print(f"psutil: {psutil.__version__}")
print(f"pandas: {pd.__version__}")
print(f"numpy: {np.__version__}")
print(f"seaborn: {sns.__version__}")
print(f"matplotlib: {matplotlib.__version__}")

---- System information ----
OS: Linux, 5.3.0-24-generic
Platform: x86_64
Logical cores: 8
Max Frequency: 4000 Mhz
Total Memory: 31.13 GB

---- Python information ----
python: 3.7.3
psutil: 5.6.7
pandas: 0.25.3
numpy: 1.17.3
seaborn: 0.9.0
matplotlib: 3.1.2


## Configuration

In [3]:
DATA_PATH = Path(".") / "data" / "alexa"

DATASETS = {
    "US": {
        "url": "https://www.ssa.gov/oact/babynames/names.zip",
        "filename": "names_us.zip",
        "release": datetime.date(2015, 6, 28),
    },
    "CH": {
        "url_prefix": "",
        "url": [
            "https://www.bfs.admin.ch/bfsstatic/dam/assets/9127668/master",
            "https://www.bfs.admin.ch/bfsstatic/dam/assets/9127688/master",
        ],
        "filename": "names_ch.zip",
        "release": datetime.date(2015, 6, 28),
    },
    "EN, WLS": {
        "url_prefix": "https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2flivebirths%2fdatasets%2fbabynamesenglandandwales",
        "url": [
            "top100babynameshistoricaldata%2f19041994/historicname_tcm77-254032.xls",
            "babynamesstatisticsboys%2f1996/1996boys_tcm77-254026.xls",
            "babynamesstatisticsboys%2f1997/1997boys_tcm77-254022.xls",
            "babynamesstatisticsboys%2f1998/1998boys_tcm77-254018.xls",
            "babynamesstatisticsboys%2f1999/1999boys_tcm77-254014.xls",
            "babynamesstatisticsboys%2f2000/2000boys_tcm77-254008.xls",
            "babynamesstatisticsboys%2f2001/2001boys_tcm77-254000.xls",
            "babynamesstatisticsboys%2f2002/2002boys_tcm77-253994.xls",
            "babynamesstatisticsboys%2f2003/2003boys_tcm77-253990.xls",
            "babynamesstatisticsboys%2f2004/2004boys_tcm77-253986.xls",
            "babynamesstatisticsboys%2f2005/2005boys_tcm77-253982.xls",
            "babynamesstatisticsboys%2f2006/2006boys_tcm77-253978.xls",
            "babynamesstatisticsboys%2f2007/2007boys_tcm77-253973.xls",
            "babynamesstatisticsboys%2f2008/2008boys_tcm77-253966.xls",
            "babynamesstatisticsboys%2f2009/2009boys_tcm77-253932.xls",
            "babynamesstatisticsboys%2f2010/2010boys_tcm77-253928.xls",
            "babynamesstatisticsboys%2f2011/2011boysbabynamesfinal_tcm77-276133.xls",
            "babynamesstatisticsboys%2f2012/2012boysbyareagorrsmonthwebtables_tcm77-323077.xls",
            "babynamesstatisticsboys%2f2013/2013boysbyareagorrsmonthwebtables_tcm77-374580.xls",
            "babynamesstatisticsboys%2f2014/2014boysbyareagorrsmonthwebtables_tcm77-413738.xls",
            "babynamesstatisticsboys%2f2015/2015boysnamesfinal.xls",
            "babynamesstatisticsboys%2f2016/2016boysnames.xls",
            "babynamesstatisticsboys%2f2017/2017boysnames.xls",
            "babynamesstatisticsboys%2f2018/2018boysnames.xls",
            "babynamesstatisticsgirls%2f1996/1996girls_tcm77-254024.xls",
            "babynamesstatisticsgirls%2f1997/1997girls_tcm77-254020.xls",
            "babynamesstatisticsgirls%2f1998/1998girls_tcm77-254016.xls",
            "babynamesstatisticsgirls%2f1999/1999girls_tcm77-254010.xls",
            "babynamesstatisticsgirls%2f2000/2000girls_tcm77-254006.xls",
            "babynamesstatisticsgirls%2f2001/2001girls_tcm77-253998.xls",
            "babynamesstatisticsgirls%2f2002/2002girls_tcm77-253992.xls",
            "babynamesstatisticsgirls%2f2003/2003girls_tcm77-253988.xls",
            "babynamesstatisticsgirls%2f2004/2004girls_tcm77-253984.xls",
            "babynamesstatisticsgirls%2f2005/2005girls_tcm77-253980.xls",
            "babynamesstatisticsgirls%2f2006/2006girls_tcm77-253976.xls",
            "babynamesstatisticsgirls%2f2007/2007girls_tcm77-253971.xls",
            "babynamesstatisticsgirls%2f2008/2008girls_tcm77-253964.xls",
            "babynamesstatisticsgirls%2f2009/2009girls_tcm77-253940.xls",
            "babynamesstatisticsgirls%2f2010/2010girls_tcm77-253930.xls",
            "babynamesstatisticsgirls%2f2011/2011girlsbabynamesfinal_tcm77-276135.xls",
            "babynamesstatisticsgirls%2f2012/2012girlsbyareagorrsmonthwebtables_tcm77-323080.xls",
            "babynamesstatisticsgirls%2f2013/2013girlsbyareagorrsmonthwebtables_tcm77-374588.xls",
            "babynamesstatisticsgirls%2f2014/2014girlsbyareagorrsmonthwebtables_tcm77-413741.xls",
            "babynamesstatisticsgirls%2f2015/2015girlsnamesfinal.xls",
            "babynamesstatisticsgirls%2f2016/2016girlsnames.xls",
            "babynamesstatisticsgirls%2f2017/2017girlsnames.xls",
            "babynamesstatisticsgirls%2f2018/2018girlsnames.xls",
        ],
        "filename": "names_en_wls.zip",
        "release": datetime.date(2016, 9, 28),
    },
}

# Preparation

## Download Datasets

In [4]:
# Create target folder, if not existing
DATA_PATH.mkdir(parents=True, exist_ok=True)

# Download and store files
for name, desc in DATASETS.items():
    print(f"Downloading {name}-dataset:")
    target_file = DATA_PATH / desc["filename"]

    if target_file.exists():
        print(f"Skipped download: {target_file} already exists. (Delete file in ./data/alexa first to re-download)")
        continue

    # Handling multiple files...
    if isinstance(desc["url"], list):
        with ZipFile(target_file, "w") as zipObj:
            for idx, url in enumerate(desc["url"]):
                url = desc["url_prefix"] + url
                print(f"Get {url[-80:]} ...")
                req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
                resp = urllib.request.urlopen(req)
                stream = io.BytesIO()
                stream.write(resp.read())
                stream.seek(0)
                zipObj.writestr(resp.info().get_filename(), stream.getvalue())
    # Handling single file...
    else:
        print(f"Get {desc['url'][-80:]}")
        urllib.request.urlretrieve(desc["url"], target_file)

    print("Done.\n")

Get https://www.ssa.gov/oact/babynames/names.zip
Done.

Get https://www.bfs.admin.ch/bfsstatic/dam/assets/9127668/master ...
Get https://www.bfs.admin.ch/bfsstatic/dam/assets/9127688/master ...
Done.

Get ndandwalestop100babynameshistoricaldata%2f19041994/historicname_tcm77-254032.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f1996/1996boys_tcm77-254026.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f1997/1997boys_tcm77-254022.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f1998/1998boys_tcm77-254018.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f1999/1999boys_tcm77-254014.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f2000/2000boys_tcm77-254008.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f2001/2001boys_tcm77-254000.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f2002/2002boys_tcm77-253994.xls ...
Get babynamesenglandandwalesbabynamesstatisticsboys%2f2003/2003boys_tcm77-253990.xls ..

**Let's check, how much space the (zipped) data needs on our Harddrive:**

In [5]:
data_size = sum(f.stat().st_size for f in DATA_PATH.glob("**/*") if f.is_file())
print(f"Size of all data files (/{DATA_PATH}/*): {data_size / 1024 / 1024:.2f} MB")

Size of all data files (/data/alexa/*): 49.86 MB


## Prepare Dataset

Helper functions for simple loading datasets into pandas DataFrame:

In [ ]:
def gen_dataset_us():
    """Generator yielding on year from US dataset.

    Yields:
        {pd.DataFrame} -- Dataframe containing names of one year

    """
    file_path = DATA_PATH / DATASETS["US"]["filename"]
    with ZipFile(file_path, "r") as zip:
        files = [f for f in zip.namelist() if f.endswith(".txt")]
        for f in files:
            df = pd.read_csv(zip.open(f), names=["Name", "Gender", "Count"])
            df["Year"] = pd.to_datetime(int(f[3:7]), format="%Y")
            df["Dataset"] = "US"
            df["Rank"] = df.groupby("Gender")["Count"].rank(
                method="max", ascending=False
            )
            yield df

In [ ]:
def gen_dataset_ch():
    """Generator yielding on year from CH dataset.

    Yields:
        {pd.DataFrame} -- Dataframe containing names of one year

    """
    file_path = DATA_PATH / DATASETS["US"]["filename"]
    with ZipFile(file_path, "r") as zip:
        files = [f for f in zip.namelist() if f.endswith(".txt")]
        for f in files:
            df = pd.read_csv(zip.open(f), names=["Name", "Gender", "Count"])
            df["Year"] = pd.to_datetime(int(f[3:7]), format="%Y")
            df["Dataset"] = "US"
            df["Rank"] = df.groupby("Gender")["Count"].rank(
                method="max", ascending=False
            )
            yield df

In [ ]:
def gen_dataset_en_wls():
    """Generator yielding on year from GB dataset.

    Yields:
        {pd.DataFrame} -- Dataframe containing names of one year

    """

    def _map_year_sheet(year):
        if year == 1996:
            return 3
        elif year <= 2010:
            return 6
        else:
            return 8

    def _map_year_header(year):
        if year <= 2008:
            return 5
        elif year <= 2016:
            return 4
        else:
            return 5

    def _map_year_cols(year):
        if year <= 2016:
            return [1, 2, 3]
        else:
            return [0, 1, 2]

    def _load_historic(zip, fn, sheet):
        df = pd.read_excel(zip.open(fn), sheet_name=sheet, header=3, skipfooter=2)
        # Drop empty first row
        df = df.dropna()
        # Unpivot
        df = df.melt(id_vars=["RANK"], value_vars=[1904])
        # Rename Columns
        df.columns = ["Rank", "Year", "Name"]
        # Normalize Casing
        df["Name"] = df["Name"].str.title().str.strip()
        # Cast Datatypes
        df["Rank"] = df["Rank"].astype(int)
        df["Year"] = pd.to_datetime(df["Year"], format="%Y")
        # Add Gender and Country information
        df["Gender"] = "M" if sheet == "Boys" else "F"
        df["Dataset"] = "EN, WLS"
        return df

    def _load_recent(zip, fn):
        year = int(fn[:4])
        df = pd.read_excel(
            zip.open(fn),
            sheet_name=_map_year_sheet(year),
            header=_map_year_header(year),
            usecols=_map_year_cols(year),
            names=["Rank", "Name", "Count"],
        )
        # Number of footer rows in xls varies, so sometimes we get empty rows
        df = df.dropna()
        # Normalize Casing
        df["Name"] = df["Name"].str.title().str.strip()
        # Cast Datatypes
        df[["Rank", "Count"]] = df[["Rank", "Count"]].astype(int)
        # Add year and gender (from filename), and country information
        df["Year"] = pd.to_datetime(year, format="%Y")
        df["Gender"] = "M" if "boys" in fn else "F"
        df["Dataset"] = "EN, WLS"
        return df

    file_path = DATA_PATH / DATASETS["EN, WLS"]["filename"]
    with ZipFile(file_path, "r") as zip:
        files = [fn for fn in zip.namelist() if fn.endswith(".xls")]
        for fn in files:
            if fn.startswith("historicname"):
                yield _load_historic(zip, fn, "Boys")
                yield _load_historic(zip, fn, "Girls")
            if fn[:4].isdigit():
                yield _load_recent(zip, fn)

In [ ]:
def load_datasets(names):
    """Helper function to load datasets into single dataframe.

    This also brings all datasets into the same form.

    Arguments:
        name {list} -- List of dataset short names, according to DATASETS

    Returns:
        {pd.DataFrame} -- All datasets in a single dataframe

    """
    list_dfs = []

    # Load individual Datasets
    if "US" in names or "all" in names:
        list_dfs.extend(gen_dataset_us())

    if "EN, WLS" in names or "all" in names:
        list_dfs.extend(gen_dataset_en_wls())

    # Merge list of dataframes into single dataframe
    df_all = pd.concat(list_dfs, axis=0, ignore_index=True, sort=False)

    return df_all

**Actually loading the data:**

In [ ]:
%%time

df = load_datasets(["all"])
display(df.head())
display(df.tail())

**Calculating percentages of names per year, dataset and gender:**

In [ ]:
def calculate_percentage(df):
    # Group by dataset and year
    # (using agg .first() as  we only have one value per group
    df = df.groupby(["Dataset", "Year", "Gender", "Name", "Rank"]).first()

    # transform count=0 back to to NaN
    df["Count"] = df["Count"].replace(0, np.nan)

    # Calculate percentage of names per year
    df["Percentage"] = 100 * df / df.groupby(level=1).transform("sum")

    # Convert multiindex back to to data column
    df = df.reset_index()
    return df

In [ ]:
%%time

df = calculate_percentage(df)
df.tail()

## Quick Sanity Checks

**Are the percentages calculated correctely?**

In [ ]:
perc_female_temp = sum(
    df[(df["Dataset"] == "US") & (df["Gender"] == "F") & (df["Year"] == "1920-01-01")][
        "Percentage"
    ]
)
perc_male_temp = sum(
    df[(df["Dataset"] == "US") & (df["Gender"] == "M") & (df["Year"] == "1920-01-01")][
        "Percentage"
    ]
)

print(f"US in 1920:\n{'-'*40}\nFemale names:\t{perc_female_temp:.2f} %")
print(f"Male names:\t{perc_male_temp:.2f} %")
print(f"Total:\t\t{perc_female_temp+perc_male_temp} %")

**Do ranks, counts and percentage make sense?**

In [ ]:
df_temp = df[(df["Name"].isin(["Alexa", "Victoria"])) & (df["Gender"] == "F")]

fig, ax = plt.subplots(3, 1, figsize=(14, 8))
sns.lineplot(x="Year", y="Count", hue="Name", style="Dataset", data=df_temp, ax=ax[0])
sns.lineplot(
    x="Year", y="Percentage", hue="Name", style="Dataset", data=df_temp, ax=ax[1]
)
sns.lineplot(x="Year", y="Rank", hue="Name", style="Dataset", data=df_temp, ax=ax[2]);

## Filter dataset

**Limiting to Name "Alexa":**

Because this is what we want to visualize here. Also dropping "Name" column, as the data now only contains a single name.

In [ ]:
df_alexa = df[(df["Name"] == "Alexa") & (df["Gender"] == "F")]

# Also drop "Name", as everything is "Alexa" now:
df_alexa = df_alexa.drop(columns="Name")

df_alexa.head()

**Limiting to years since 1996:** 

We don't have complete dataset for all countries earlier, see Sanity Checks.

In [ ]:
df_alexa = df_alexa[(df_alexa["Year"] >= "1996-01-01")]
df_alexa.head()

**TODO: Wrap whole preprocessing**

# Visualization

## Quick and dirty version

In [ ]:
datasets_count = len(DATASETS)
fig, ax = plt.subplots(datasets_count, 2, figsize=(16, datasets_count * 4))

for idx, dataset in enumerate(DATASETS.keys()):
    df_alexa_dataset = df_alexa[df_alexa["Dataset"] == dataset]
    sns.lineplot(
        x="Year", y="Percentage", data=df_alexa_dataset, ax=ax[idx][0], marker="o"
    )
    sns.lineplot(x="Year", y="Rank", data=df_alexa_dataset, ax=ax[idx][1], marker="o")
    ax[idx][1].invert_yaxis()

    ax[idx][0].axvline(x=DATASETS[dataset]["release"], color="orangered", linewidth=1)
    ax[idx][1].axvline(x=DATASETS[dataset]["release"], color="orangered", linewidth=1);

## Final Version

# References

[1] Wikipedia contributors (2019): **Amazon Alexa**. In *Wikipedia - The Free Encyclopedia*. Retrieved Dec 28, 2019, from <https://en.wikipedia.org/w/index.php?title=Amazon_Alexa&oldid=932280905>

[2] Office for National Statistics (2019): **Baby names in England and Wales: 1996-2018**. Retrieved Dec 28, 2019, from <https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/datasets/babynamesenglandandwalesbabynamesstatisticsgirls> and <https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/datasets/babynamesenglandandwalesbabynamesstatisticsboys>

[3] Office for National Statistics (2019): **Top 100 baby names in England and Wales: 1907-1994**. Retrieved Dec 28, 2019, from <https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/datasets/babynamesenglandandwalestop100babynameshistoricaldata>

[4] Social Security Administration (2019): **Baby Names from Social Security Card Applications - National Data**. Retrieved Jan 03, 2020, from https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-national-level-data